# 7.2. Площади

В этой задаче Вам требуется сравнить значения метрик для 4 различных алгоритмов. Для этого отредактируйте следующий код так, чтобы он соответствовал сформулированному заданию.

In [1]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve


X, y = load_breast_cancer(return_X_y=True)
X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.fit_transform(X_train)
x_test_scaled = scaler.fit_transform(x_test)

classifiers = [
    DecisionTreeClassifier(random_state=42),
    LogisticRegression(random_state=42),
    KNeighborsClassifier(n_neighbors=5),
    SVC(probability=True, random_state=42),
]

roc_auc_scores = np.empty(len(classifiers), dtype=np.float64)
pr_auc_scores = np.empty(len(classifiers), dtype=np.float64)

for i, classifier in enumerate(classifiers):
    classifier.fit(X_train_scaled, y_train)
    y_pred_proba = classifier.predict_proba(x_test_scaled)[:, 1]
    roc_auc_scores[i] = roc_auc_score(y_test, y_pred_proba)

    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    pr_auc_scores[i] = auc(recall, precision)

print(roc_auc_scores, pr_auc_scores)
print(roc_auc_scores.argmax() + 1, pr_auc_scores.argmax() + 1)

[0.92791005 0.99794239 0.98052616 0.99588477] [0.96447552 0.99875396 0.98754243 0.99741534]
2 2


## Примечания

1. Обратите внимание, что StandardScaler может влиять на результаты работы алгоритмов. Поэтому рекомендуем его использовать.

2. Когда мы разбиваем данные на train и test, мы должны понимать, что **все** алгоритмы должны обучаться **только на train**. Test при обучении **не используется**. Более того, в реальной жизни мы вообще ничего не знаем про test. Поэтому StandardScaler нужно обучать только на X_train, а к X_test делать только transform (**не** fit_transform).